From the previous implementation of our wavenet we left out how the conditonal and global outputs would be added together. Now we can fill in that part with resized contional inputs

In [1]:
import torch
import torchaudio.datasets as datasets
from model import WaveNet

In [2]:
dataset = datasets.LJSPEECH("../data")

In [3]:
waveform, sample_rate, transcript, normalized_transcript = dataset[0]

waveform, sample_rate, normalized_transcript

(tensor([[-7.3242e-04, -7.6294e-04, -6.4087e-04,  ...,  7.3242e-04,
           2.1362e-04,  6.1035e-05]]),
 22050,
 'Printing, in the only sense with which we are at present concerned, differs from most if not from all the arts and crafts represented in the Exhibition')

In [4]:
local_inputs = torch.randn((1,5,6))
local_inputs.shape, local_inputs.size(1)

(torch.Size([1, 5, 6]), 5)

In [5]:
model = WaveNet(2, 2, 2, local_channels=local_inputs.size(1))

In [6]:
probs = model(waveform.view(1, 1, -1), local_inputs)
probs.shape

[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.


torch.Size([1, 256, 1])

In [7]:
from preprocess import decodeMuLaw

In [8]:
classification = torch.argmax(probs)
predicted_output = decodeMuLaw(classification)
classification, predicted_output

(tensor(167), tensor(0.0175))

In [9]:
import torch.nn.functional as f

In [10]:
waveforms = [dataset[i][0] for i in range(3)]

In [11]:
max_size = max(list(map(lambda x: x.size(-1), waveforms)))
max_size

213149

In [12]:
f.pad(waveforms[0], (0, max_size-waveforms[0].size(-1))).shape

torch.Size([1, 213149])

In [13]:
padded_waveforms = list(map(lambda x: f.pad(x, (0, max_size-x.size(-1))), waveforms))
inputs = torch.stack(padded_waveforms)
inputs.shape

torch.Size([3, 1, 213149])

In [14]:
repeated_local_inputs = local_inputs.repeat(3,1,1)
repeated_local_inputs.shape

torch.Size([3, 5, 6])

In [15]:
model(inputs, repeated_local_inputs).shape

torch.Size([3, 256, 1])